# IMU Orientation Tracking

**Learning Objective**: Understand IMU drift and the challenges of integration-based tracking

**Estimated Time**: 12 minutes

This notebook demonstrates why IMUs alone cannot provide accurate orientation over time due to drift accumulation.

In [ ]:
# Install dependencies
!pip install numpy==1.24.0 matplotlib==3.7.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In [ ]:
def simulate_angular_velocity(
    time: np.ndarray,
    true_rotation_rate: float = 0.5
) -> np.ndarray:
    """
    Simulate gyroscope measurements (angular velocity in rad/s).
    """
    omega_true = np.full_like(time, true_rotation_rate)
    noise = np.random.normal(0, 0.02, size=omega_true.shape)
    drift = 0.01 * np.sin(0.1 * time)  # Slow drift
    return omega_true + noise + drift

def integrate_orientation(
    omega: np.ndarray,
    dt: float,
    initial_angle: float = 0.0
) -> np.ndarray:
    """
    Integrate angular velocity to get orientation (Euler integration).
    """
    angles = np.zeros(len(omega))
    angles[0] = initial_angle
    
    for i in range(1, len(omega)):
        angles[i] = angles[i-1] + omega[i-1] * dt
    
    return angles

def calculate_true_orientation(time: np.ndarray, rotation_rate: float) -> np.ndarray:
    """Calculate ground truth orientation."""
    return rotation_rate * time

def calculate_drift_metrics(
    true_angles: np.ndarray,
    measured_angles: np.ndarray,
    time: np.ndarray
) -> dict:
    """Calculate drift metrics."""
    error = measured_angles - true_angles
    return {
        'final_error_deg': np.rad2deg(abs(error[-1])),
        'mean_error_deg': np.rad2deg(np.mean(np.abs(error))),
        'drift_rate_deg_per_sec': np.rad2deg(abs(error[-1])) / time[-1]
    }

In [ ]:
# Simulation parameters
duration = 30.0
dt = 0.01  # 100 Hz
time = np.arange(0, duration, dt)
rotation_rate = 0.5  # rad/s

# Simulate gyroscope and integrate
omega_measured = simulate_angular_velocity(time, rotation_rate)
measured_angles = integrate_orientation(omega_measured, dt)
true_angles = calculate_true_orientation(time, rotation_rate)

# Calculate metrics
metrics = calculate_drift_metrics(true_angles, measured_angles, time)

print("=" * 60)
print("IMU ORIENTATION TRACKING: The Drift Problem")
print("=" * 60)
print("\nDrift Metrics (after 30 seconds):")
print(f"  • Final Error: {metrics['final_error_deg']:.2f}°")
print(f"  • Mean Absolute Error: {metrics['mean_error_deg']:.2f}°")
print(f"  • Drift Rate: {metrics['drift_rate_deg_per_sec']:.3f}°/second")
print("\n" + "=" * 60)
print("KEY INSIGHT: Integration causes error accumulation!")
print("Solution: Sensor fusion (combine IMU with cameras/GPS)")
print("=" * 60)

In [ ]:
# Visualize drift
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Orientation comparison
ax1.plot(time, np.rad2deg(true_angles), 'g-', linewidth=2, label='True Orientation', alpha=0.7)
ax1.plot(time, np.rad2deg(measured_angles), 'r--', linewidth=1.5, label='IMU Estimate', alpha=0.8)
ax1.set_xlabel('Time (seconds)', fontsize=11)
ax1.set_ylabel('Orientation (degrees)', fontsize=11)
ax1.set_title('IMU Orientation Tracking: The Drift Problem', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Error accumulation
error = np.abs(measured_angles - true_angles)
ax2.plot(time, np.rad2deg(error), 'b-', linewidth=2, label='Absolute Error')
ax2.fill_between(time, 0, np.rad2deg(error), alpha=0.3)
ax2.set_xlabel('Time (seconds)', fontsize=11)
ax2.set_ylabel('Error (degrees)', fontsize=11)
ax2.set_title('Error Accumulation Over Time', fontsize=12, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Expected Output

You should see:
1. **Metrics**: Final error grows to ~20-30°, drift rate ~1°/sec
2. **Visualization**: Growing gap between true and IMU-estimated orientation

**Key Takeaway**: IMU drift is a fundamental challenge in Physical AI. Robots must fuse IMU data with other sensors (cameras, GPS) to correct accumulated errors.